In [3]:
import torch
import os
import imageio

from model import *
from dataset import load_blender_data
from run import config_parser, create_nerf, render_path

import warnings
warnings.filterwarnings(action='ignore')

os.environ['CUDA_VISIBLE_DEVICES']='0'

Testing

In [4]:
parser = config_parser()
parser_str = '--config config_lego.txt --render_only'
args, _ = parser.parse_known_args(args=parser_str.split())
args.lindisp = False
render_kwargs_train, render_kwargs_test, start, grad_vars, optimizer = create_nerf(args)

Found ckpts ['./data/nerf_synthetic/nerf/fox\\blender_paper_lego\\200000.tar']
Reloading from ./data/nerf_synthetic/nerf/fox\blender_paper_lego\200000.tar
Not ndc!


In [ ]:
render_kwargs_train['network_fn']

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
np.random.seed(0)

def render():

    parser = config_parser()
    parser_str = '--config config_lego.txt --render_only'
    args, _ = parser.parse_known_args(args=parser_str.split())
    args.lindisp = False
    
    print(args)

    # Load data
    K = None

    images, poses, render_poses, hwf, i_split = load_blender_data(args.datadir, args.half_res, args.testskip)
    print('Loaded blender', images.shape, render_poses.shape, hwf, args.datadir)
    i_train, i_val, i_test = i_split

    near = 2.
    far = 6.

    if args.white_bkgd:
        images = images[...,:3]*images[...,-1:] + (1.-images[...,-1:])
    else:
        images = images[...,:3]

    # Cast intrinsics to right types
    H, W, focal = hwf
    H, W = int(H), int(W)
    hwf = [H, W, focal]

    if K is None:   
        K = np.array([
            [focal, 0, 0.5*W],
            [0, focal, 0.5*H],
            [0, 0, 1]
        ])

    # Create log dir and copy the config file
    basedir = args.basedir
    expname = args.expname
    os.makedirs(os.path.join(basedir, expname), exist_ok=True)
    f = os.path.join(basedir, expname, 'args.txt')
    with open(f, 'w') as file:
        for arg in sorted(vars(args)):
            attr = getattr(args, arg)
            file.write('{} = {}\n'.format(arg, attr))
    if args.config is not None:
        f = os.path.join(basedir, expname, 'config.txt')
        with open(f, 'w') as file:
            file.write(open(args.config, 'r').read())

    # Create nerf model
    render_kwargs_train, render_kwargs_test, start, grad_vars, optimizer = create_nerf(args)
    global_step = start

    bds_dict = {
        'near' : near,
        'far' : far,
    }
    render_kwargs_train.update(bds_dict)
    render_kwargs_test.update(bds_dict)

    # Move testing data to GPU
    render_poses = torch.Tensor(render_poses).to(device)

    # Short circuit if only rendering out from trained model
    print('RENDER ONLY')
    with torch.no_grad():
        # Default is smoother render_poses path
        images = None

        testsavedir = 'result'
        os.makedirs(testsavedir, exist_ok=True)
        print('test poses shape', render_poses.shape)
        
        # Rendering
        rgbs, _ = render_path(render_poses, hwf, K, args.chunk, render_kwargs_test, gt_imgs=images, savedir=testsavedir, render_factor=args.render_factor)
        print('Done rendering', testsavedir)
        imageio.mimwrite(os.path.join(testsavedir, 'video.mp4'), to8b(rgbs), fps=30, quality=8)

In [ ]:
render()